<a href="https://colab.research.google.com/github/cagBRT/promptEngineering/blob/main/4_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/promptEngineering.git cloned-repo
%cd cloned-repo

In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [ ]:
!pip install xformers

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
from pprint import pprint
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
from transformers import TFAutoModelForMaskedLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
from torch import nn


# Summary of the tasks
This notebook shows some of the most frequent use-cases NLP transformers.

Tasks demonstrated in this notebook:
>question answering<br>
sequence classification<br>
extractive question answering<br>
language modeling<br>
masked language modeling<br>
causal language modeling<br>
text generation<br>
named entity recognition<br>
checking paraphrasing<br>

audio classification<br>
automatic speech recognition<br>
image classification<br>

These examples use auto-models, which are classes that will instantiate a model according to a given checkpoint, automatically selecting the correct model architecture.

**For a model to perform well on a task, it must be loaded from a checkpoint corresponding to that task.** <br><br>
These checkpoints are usually pre-trained on a large corpus of data and fine-tuned on a specific task. This means the following:

- Not all models were fine-tuned on all tasks. If you want to fine-tune a model on a specific task, you can leverage one of the run_$TASK.py scripts in the examples directory.<vbr>

- Fine-tuned models were fine-tuned on a specific dataset. This dataset may or may not overlap with your use-case and domain. As mentioned previously, you may leverage the examples scripts to fine-tune your model, or you may create your own training script.<br>

**In order to do an inference on a task, several mechanisms are made available by the library:**

- Pipelines: very easy-to-use abstractions, which require as little as two lines of code.
- Direct model use: Less abstractions, but more flexibility and power via a direct access to a tokenizer (PyTorch/TensorFlow) and full inference capacity.
Both approaches are demonstrated in this notebook.



All tasks presented here leverage pre-trained checkpoints that were fine-tuned on specific tasks. Loading a checkpoint that was not fine-tuned on a specific task would load only the base transformer layers and not the additional head that is used for the task, initializing the weights of that head randomly. This would produce random output.

# Sequence Classification

**Sequence classification is the task of classifying sequences according to a given number of classes.** <br>

In this example we use pipelines to do sentiment analysis: identifying if a sequence is positive or negative. It leverages a fine-tuned model on sst2, which is a GLUE task.

The model returns a label ("POSITIVE" or "NEGATIVE") alongside a score, as follows:


Notice the code does not specify a model- we are using "pipeline abstraction" to do sentiment-analysis. <br>
For production release, you will need to specify a model

In [ ]:
#from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier("I hate you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

In [ ]:
result = classifier("I love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")



---



---



---



#Checking for Paraphrases

Here is an example of doing a sequence classification using a model to determine if two sequences are paraphrases of each other.

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it with the weights stored in the checkpoint.<br>
2. Build a sequence from the two sentences, with the correct model-specific separators, token type ids and attention masks (which will be created automatically by the tokenizer).<br>
3. Pass this sequence through the model so that it is classified in one of the two available classes:
>0 (not a paraphrase) <br>
1 (is a paraphrase)<br>

Compute the softmax of the result to get probabilities over the classes.
Print the results.

**TensorFlow**

In [ ]:
#from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
#import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# The tokenizer will automatically add any model specific separators (i.e.  and ) and tokens to
# the sequence, as well as compute the attention masks.

#====
#compare senquence0 and sequence2
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="tf")
#compare senquence0 and sequence1
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="tf")

#logits can be defined as a vector of raw predictions that a
#classification model generates and it is passed to a normalized function.

paraphrase_classification_logits = model(paraphrase).logits
not_paraphrase_classification_logits = model(not_paraphrase).logits

paraphrase_results = tf.nn.softmax(paraphrase_classification_logits, axis=1).numpy()[0]
not_paraphrase_results = tf.nn.softmax(not_paraphrase_classification_logits, axis=1).numpy()[0]

In [ ]:
# Should be paraphrased
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

In [ ]:
# Should not be paraphrased
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

**Assignment**<br>
Write different sentences, some paraphrasing, others not. <br>
See how well the model performs



---


---




---



# Extractive Question Answering
Extractive Question Answering is the task of extracting an answer from a text given a question.<br>
 An example of a question answering dataset is the [SQuAD dataset](https://rajpurkar.github.io/SQuAD-explorer/), which is entirely based on that task. If you would like to fine-tune a model on a SQuAD task, you may leverage the run_qa.py and run_tf_squad.py scripts.



**Stanford Question Answering Dataset (SQuAD)** is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles,

*where the answer to every question is a segment of text, or span, from the corresponding reading passage,*

or the question might be unanswerable.

Here is an example of using pipelines to do question answering: extracting an answer from a text given a question. <br>
It leverages a  model fine-tuned on SQuAD.

In [ ]:
#from transformers import pipeline

question_answerer = pipeline("question-answering")

context = r"""
Knitting is a method for production of textile fabrics by interlacing yarn loops with /
loops of the same or other yarns./
It is used to create many types of garments. Knitting may be done by hand or by machine./
Knitting creates stitches: loops of yarn in a row, either flat or in the round (tubular). /
There are usually many active stitches on the knitting needle at one time. /
Knitted fabric consists of a number of consecutive rows of connected loops that /
intermesh with the next and previous rows. As each row is formed, each newly created/
 loop is pulled through one or more loops from the prior row and placed on the gaining/
  needle so that the loops from the prior row can be pulled off the other needle /
  without unraveling.

Differences in yarn (varying in fibre type, weight, uniformity and twist), needle size,/
 and stitch type allow for a variety of knitted fabrics with different properties, /
 including color, texture, thickness, heat retention, water resistance, and integrity./
  A small sample of knitwork is known as a swatch.
"""

This returns an answer extracted from the text, a confidence score, alongside "start" and "end" values, which are the positions of the extracted answer in the text.



In [ ]:
textlisting="Knitting is a method for production of textile fabrics by interlacing yarn loops with loops of the same or other yarns"
textlisting[58:69]

In [ ]:
result = question_answerer(question="What is knitting", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

In [ ]:
result = question_answerer(question="What is a swatch?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)



---



---



**Here is an example of question answering using a model and a tokenizer**.

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it with the weights stored in the checkpoint.
2. Define a text and a few questions.
Iterate over the questions and build a sequence from the text and the current question, with the correct model-specific separators, token type ids and attention masks.
3. Pass this sequence through the model. This outputs a range of scores across the entire sequence tokens (question and text), for both the start and end positions.
4. Compute the softmax of the result to get probabilities over the tokens.
Fetch the tokens from the identified start and stop values, convert those tokens to a string.
5. Print the results.<br>

An answer of CLS == answer not found



---

---





---



# Language Modeling
**Language modeling is the task of fitting a model to a corpus**, which can be domain specific.

<br>All popular transformer-based models are trained using a variant of language modeling, <br>
e.g. <br>
>BERT with masked language modeling, <br>
GPT-2 with causal language modeling.

Language modeling can be useful outside of pretraining as well, <br>
For example to shift the model distribution to be domain-specific:<br>
 1. use a language model trained over a very large corpus,
 2. then fine-tune it to a news dataset or on scientific papers



# Masked Language Modeling<br>

**Masked language models do not require labelled data!**<br>
They are trained by masking a couple of words in sentences and the model is expected to guess the masked word.

<br>

Masked language modeling is the task of masking tokens in a sequence with a masking token, and **prompting the model to fill that mask with an appropriate token**.<br>

 This allows the model to attend to both the right context (tokens on the right of the mask) and the left context (tokens on the left of the mask).
 <br><br>
 **Such a training creates a strong basis for downstream tasks requiring bi-directional context**, such as SQuAD.



**Masked language** modeling is the task of masking some of the words in a sentence and predicting which words should replace those masks. These models are useful when we want to get a statistical understanding of the language in which the model is trained in.



**Use cases for Masking**: <br>
**Domain Adaptation**: train large models for domain-specific problems. If you have to work on a domain-specific task, such as retrieving information from medical research papers, you can train a masked language model using those papers. <br><br>
The resulting model has a statistical understanding of the language used in medical research papers, and can be *further trained in a process called fine-tuning to solve different tasks, such as Text Classification or Question Answering to build a medical research papers information extraction system.*<br>Pre-training on domain-specific data tends to yield better results.
<br><br>
**Inference with Fill-Mask Pipeline**
You can use the HuggingFace Transformers library fill-mask pipeline to do inference with masked language models. If a model name is not provided, the pipeline will be initialized with distilroberta-base. You can provide masked text and it will return a list of possible mask values ​​ranked according to the score.

Here is an example of using pipelines to replace a mask from a sequence:

In [ ]:
#from transformers import pipeline
unmasker = pipeline("fill-mask")

This outputs the sequences with the mask filled, the confidence score, and the token id in the tokenizer vocabulary:



**pprint stands for pretty printer**
<br>The pprint module's functionality enables aesthetically good looking appearance of Python data structures. Any data structure that can be correctly parsed by Python interpreter is elegantly formatted.

In [ ]:
#from pprint import pprint
pprint(
    unmasker(
        f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."
    )
)
#Prints top 5 possible values

In [ ]:
pprint(
    unmasker(
        f"John Deere manufactures {unmasker.tokenizer.mask_token} that farmers use to solve problems."
    )
)

**Example of doing masked language modeling using a model and a tokenizer**.


1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a DistilBERT model and loads it with the weights stored in the checkpoint.
2. Define a sequence with a masked token, placing the tokenizer.mask_token instead of a word.
3. Encode that sequence into a list of IDs and find the position of the masked token in that list.
4. Retrieve the predictions at the index of the mask token: this tensor has the same size as the vocabulary, and the values are the scores attributed to each token. **The model gives higher score to tokens it deems probable in that context.**
5. Retrieve the top 5 tokens using the PyTorch topk or TensorFlow top_k methods.
6. Replace the mask token by the tokens and print the results

In [ ]:
from transformers import TFAutoModelForMaskedLM, AutoTokenizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = TFAutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="tf")
mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = tf.math.top_k(mask_token_logits, 5).indices.numpy()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))




---



---



---



# Causal Language Modeling
**Causal language modeling is the task of predicting the token following a sequence of tokens.**

In this situation, the model only attends to the left context (tokens on the left of the mask). Such a training is particularly interesting for generation tasks.

Usually, the next token is predicted by sampling from the logits of the last hidden state the model produces from the input sequence.





**Example of using the tokenizer and model** and leveraging the top_k_top_p_filtering() method to sample the next token following an input sequence of tokens.

**This model outputs a (hopefully) coherent next token following the original sequence, which in our case is the word is or features.**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"The Raiders football team moved to Las Vegas and"

inputs = tokenizer(sequence, return_tensors="pt")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
probs = nn.functional.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)

generated = torch.cat([input_ids, next_token], dim=-1)

resulting_string = tokenizer.decode(generated.tolist()[0])
print(resulting_string)



---



---



---



# Text Generation
In text generation (a.k.a open-ended text generation) the goal is to create a coherent portion of text that is a continuation from the given context. <br><br>
The next example shows how GPT-2 can be used in pipelines to generate text. <br><br>
As a default all models apply Top-K sampling when used in pipelines, as configured in their respective configurations (see gpt-2 config for example).

In [ ]:
from transformers import pipeline

text_generator = pipeline("text-generation")
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=False))


Here, the model generates a random text with a total maximal length of 50 tokens from context "As far as I am concerned, I will". Behind the scenes, the pipeline object calls the method PreTrainedModel.generate() to generate text. The default arguments for this method can be overridden in the pipeline, as is shown above for the arguments max_length and do_sample.

Below is an example of text generation using XLNet and its tokenizer, which includes calling generate() directly:

In [ ]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer

model = TFAutoModelForCausalLM.from_pretrained("xlnet-base-cased")
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing.   """

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="tf")["input_ids"]

prompt_length = len(tokenizer.decode(inputs[0]))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1 :]

print(generated)


Text generation is currently possible with GPT-2, OpenAi-GPT, CTRL, XLNet, Transfo-XL and Reformer in PyTorch and for most models in Tensorflow as well. As can be seen in the example above XLNet and Transfo-XL often need to be padded to work well. GPT-2 is usually a good choice for open-ended text generation because it was trained on millions of webpages with a causal language modeling objective.

**Named Entity Recognition**
Named Entity Recognition (NER) is the task of classifying tokens according to a class, for example, identifying a token as a person, an organisation or a location.<br>
An example of a named entity recognition dataset is the CoNLL-2003 dataset, which is entirely based on that task. If you would like to fine-tune a model on an NER task, you may leverage the run_ner.py script.

Here is an example of using pipelines to do named entity recognition, specifically, trying to identify tokens as belonging to one of 9 classes:<br>

- O, Outside of a named entity<br>
- B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity<br>
- I-MIS, Miscellaneous entity<br>
- B-PER, Beginning of a person's name right after another person's name<br>
- I-PER, Person's name<br>
- B-ORG, Beginning of an organisation<br> right after another organisation<br>
- I-ORG, Organisation<br>
- B-LOC, Beginning of a location right after another location<br>
- I-LOC, Location<br>

It leverages a fine-tuned model on CoNLL-2003, fine-tuned by @stefan-it from dbmdz.

CoNLL-2003 is a named entity recognition dataset released as a part of CoNLL-2003 shared task: language-independent named entity recognition. <br>
The data consists of eight files covering two languages: English and German. <br>
For each of the languages there is:<br>
> a training file<br>
a development file<br>
a test file<br>
a large file with unannotated data.

In [ ]:
from transformers import pipeline

ner_pipe = pipeline("ner")

sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO,
therefore very close to the Manhattan Bridge which is visible from the window."""

This outputs a list of all words that have been identified as one of the entities from the 9 classes defined above. Here are the expected results:

In [ ]:
for entity in ner_pipe(sequence):
    print(entity)

Note how the tokens of the sequence "Hugging Face" have been identified as an organisation, and "New York City", "DUMBO" and "Manhattan Bridge" have been identified as locations.


**Named Entity Recognition**, using a model and a tokenizer. <br><br>
The process is the following:<br>

- Instantiate a tokenizer and a model from the checkpoint name. <br>
- The model is identified as a BERT model and loads it with the weights stored in the checkpoint.<br>
- Define a sequence with known entities, such as "Hugging Face" as an organisation and "New York City" as a location.<br>
- Split words into tokens so that they can be mapped to predictions. We use a small hack by, first, completely encoding and decoding the sequence, so that we're left with a string that contains the special tokens.<br>
- Encode that sequence into IDs (special tokens are added automatically).<br>
- Retrieve the predictions by passing the input to the model and getting the first output.<br>

This results in a distribution over the 9 possible classes for each token.
<br>We take the argmax to retrieve the most likely class for each token.
<br>Zip together each token with its prediction and print it.

In [ ]:
from transformers import TFAutoModelForTokenClassification, AutoTokenizer
import tensorflow as tf

model = TFAutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = (
    "Hugging Face Inc. is a company based in New York City."
    "Its headquarters are in DUMBO, "
    "therefore very close to the Manhattan Bridge."
)

inputs = tokenizer(sequence, return_tensors="tf")
tokens = inputs.tokens()

outputs = model(**inputs)[0]
predictions = tf.argmax(outputs, axis=2)

This outputs a list of each token mapped to its corresponding prediction.<br> Differently from the pipeline, here every token has a prediction as we didn't remove **the "0"th class, which means that no particular entity was found on that token**.<br>



Predictions is an integer that corresponds to the predicted class. <br>
We can use the model.config.id2label property in order to recover the class name corresponding to the class number.

**[CLS] stands for classification**. It is added at the beginning because the training tasks here is sentence classification. And because they need an input that can represent the meaning of the entire sentence, they introduce a new tag.<br><br>

They can’t take any other word from the input sequence, because the output of that is the word representation. So they add a tag that has no other purpose than being a sentence-level representation for classification.

BERT was pretrained using the format [CLS] sen A [SEP] sen B [SEP]. It is necessary for the Next Sentence Prediction task : determining if sen B is a random sentence with no links with A or not.

In [ ]:
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

**Summarization**<br>
Summarization is the task of summarizing a document or an article into a shorter text.
<br>
An example of a summarization dataset is the CNN / Daily Mail dataset, which consists of long news articles and was created for the task of summarization. If you would like to fine-tune a model on a summarization task, various approaches are described in this document.<br>





In this example the pipelines do summarization.<br>
It leverages a Bart model that was fine-tuned on the CNN / Daily Mail data set.

**The CNN / DailyMail Dataset** is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. The current version supports both extractive and abstractive summarization.<br><br>

The data consists of news articles and highlight sentences. In the question answering setting of the data, the articles are used as the context and entities are hidden one at a time in the highlight sentences.<br><br>
This produce Cloze style questions where **the goal of the model is to correctly guess which entity in the context has been hidden in the highlight**. <br><br>
In the summarization setting, the highlight sentences are concatenated to form a summary of the article.



In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

Because the summarization pipeline depends on the PreTrainedModel.generate() method, we can override the default arguments of PreTrainedModel.generate() directly in the pipeline for max_length and min_length as shown below.<br>
**The output summary:**

In [ ]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

Here is an example of doing summarization using a model and a tokenizer.<br> <br>
The process is the following:<br>

- Instantiate a tokenizer and a model from the checkpoint name.<br>
- Summarization is usually done using an encoder-decoder model, such as Bart or T5.<br>
- Define the article that should be summarized.<br>
- Add the T5 specific prefix "summarize: ".
- Use the PreTrainedModel.generate() method to generate the summary.


In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="tf", max_length=512)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))